In [1]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [2]:
#hide
from nbdev.showdoc import *
import src.setEnv 

In [3]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time
import json

In [4]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'bf2f5aac4a0613e77432b5875bbf985de9645187b595376cb9d3f8796dccc3eb:c5260b132a8d4f8f8c76672e31db3d3d'

## User Input Class

In [5]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    email: str 
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed,
                email = self.email
            )
        userTable.save()
        

In [6]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123', 'email': 'chanopsethiwan233@gmail.com'})
example

UserInput(username='DelG', password='123', email='chanopsethiwan233@gmail.com')

## Main Lambda Handler

In [7]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    class CountUsernameError(Exception): pass
    class UsernameAlreadyExistError(Exception): pass
    class CountEmailError(Exception):pass
    class EmailAlreadyExistError(Exception):pass
    @classmethod
    @beartype
    def parseInput(cls, event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            count = UserTable.username_index.count(user.username)
        except Exception as e:
            raise cls.CountUsernameError(e)
        try: 
            countEmail = UserTable.email_index.count(user.email)
        except Exception as e:
            raise cls.CountEmailError(e)
        if not count:
            if not countEmail:
                try:
                    user.saveTable()
                    return True
                except Exception as e:
                    raise cls.SavingError(e)
            else:
                raise cls.EmailAlreadyExistError
        else:
            raise cls.UsernameAlreadyExistError
    
        

In [8]:
#export
def signUp(event, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except H.CountUsernameError as e:
        return Response.returnError(f'failed to count number of user {e}')
    except H.UsernameAlreadyExistError as e:
        return Response.returnError(f'username already exist {e}')
    except H.CountEmailError as e:
        return Response.returnError(f'failed to count number of emails {e}')
    except H.EmailAlreadyExistError as e:
        return Response.returnError(f'email already exist {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [9]:
input_ = {'username': 'Delg456', 'password': '12765', 'email': 'chanopsethiwan233@gmail.com'}
event = Event.getInput(body = input_)
signUp(event)

{'body': '{"error":"failed to count number of emails Failed to query items: An error occurred (ValidationException) on request (NGU1O72GMB8BQB11O8B3L1QKO7VV4KQNSO5AEMVJF66Q9ASUAAJG) on table (usertablechap-test-master) when calling the Query operation: ExpressionAttributeValues contains invalid value: The parameter cannot be converted to a numeric value: chanopsethiwan233@gmail.com for key :0"}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [10]:
epoch = time()
epoch

1660993675.3556201

In [11]:
a = Event.getInput({'username': 'DelG197', 'password': '123456'})
json.dumps(a)

'{"body": "{\\"username\\":\\"DelG197\\",\\"password\\":\\"123456\\"}", "httpMethod": null, "multiValueHeaders": null, "multiValueQueryStringParameters": null, "path": null, "pathParameters": null, "queryStringParameters": {}, "requestContext": null, "headers": {}, "statusCode": 200, "isBase64Encoded": false}'

In [12]:
a = ''
if not a:
    print('false')
else:
    print('true')

false


In [13]:
print(not False)

True
